In [5]:
from gdb2csv import converter
from gdbList import list_layers_with_types
import pandas as pd
import geopandas as gpd
import psycopg2
import leafmap

In [16]:
# Define the paths and layer name (comment out either one gdb_path when not in use)
#gdb_path = "data/iB1000.gdb"
gdb_path = r"C:\Users\Steve_Lau\Desktop\LS Training\ls_project1\iB1000.gdb"
csv_path1 = "data/BUILDINGNAME.csv"
layer_name1 = "BUILDINGNAME"
csv_path2 = "data/BUILDINGRELATEBUILDINGNAME.csv"
layer_name2 = "BUILDINGRELATEBUILDINGNAME"


In [17]:
try:
    conn = psycopg2.connect("dbname='sdb' user='postgres' password='Cwlau69095500' host='localhost'")

except:
    print("I am unable to connect to the database")

In [27]:
buildingname = gpd.read_postgis("SELECT *, building.shape FROM building", conn)

ValueError: Query missing geometry column 'geom'

In [4]:
# List all layers within GDB by calling "list_layers_with_types" function
list_layers_with_types(gdb_path)

Layers in the GeoDatabase:
Layer: UtilityNumber, Type: None
Layer: BUILDINGRELATEBUILDINGNAME, Type: None
Layer: SITENAME, Type: None
Layer: SITERELATESITENAME, Type: None
Layer: Street_Code, Type: None
Layer: BUILDINGNAME, Type: None
Layer: ADDRESSRELATEBUILDINGSITE, Type: None
Layer: ADDRESS, Type: None
Layer: BSLineAnno, Type: Multi Polygon
Layer: BSPointAnno, Type: Multi Polygon
Layer: BSPolygonAnno, Type: Multi Polygon
Layer: Building, Type: 3D Multi Polygon
Layer: BuildingAccessoryLine, Type: 3D Multi Line String
Layer: BuildingAnno, Type: Multi Polygon
Layer: BuildingAnnoHouseNo, Type: Multi Polygon
Layer: BuiltStructureLine, Type: 3D Multi Line String
Layer: BuiltStructurePoint, Type: 3D Point
Layer: BuiltStructurePolygon, Type: 3D Multi Polygon
Layer: CartoBuildingLine, Type: 3D Multi Line String
Layer: ProposedInfraAnno, Type: Multi Polygon
Layer: ProposedInfraStructureLine, Type: 3D Multi Line String
Layer: ProposedInfraStructurePoint, Type: 3D Point
Layer: Site, Type: 3D Mu

In [5]:
# Convert from GDB to CSV by calling "converter" function
converter(gdb_path, csv_path1, layer_name1)
converter(gdb_path, csv_path2, layer_name2)

Conversion completed successfully
Conversion completed successfully


In [6]:
# Read the CSV files into DataFrames
df1 = pd.read_csv(csv_path1)
df2 = pd.read_csv(csv_path2)

# Select only the "BUILDINGNAMEID", "ENGLISHBUILDINGNAME", and "CHINESEBUILDINGNAME" columns from df1
df1_subset = df1[["BUILDINGNAMEID", "ENGLISHBUILDINGNAME", "CHINESEBUILDINGNAME"]]

# Merge df2 with the subset of df1 on the "BUILDINGNAMEID" column
df3 = pd.merge(df2, df1_subset, on="BUILDINGNAMEID")

# Display the first few rows of the merged DataFrame
df3.head()

,LASTUPDATEDATE,BUILDINGNAMEID,BUILDINGID,ENGLISHBUILDINGNAME,CHINESEBUILDINGNAME
0,2023/09/03 00:00:00,1101367973,1100000458,Chuen Lung Village Aqua Privy,川龍村旱廁
1,2023/09/03 00:00:00,1101367804,1100004056,NaN,木棉下村公所何氏家祠
2,2023/09/03 00:00:00,1101368243,1100011094,Mount Haven Block 5,曉峰園第５座
3,2023/09/03 00:00:00,1101367784,1100003706,Yi Pei Chun Toilet,二陂圳廁所
4,2023/09/03 00:00:00,1101365720,1100002338,Lo Wai Village Office,老圍村公所


In [7]:
gdf1 = gpd.read_file(gdb_path, layer="Building")
gdf1.head()

,LASTUPDATEDATE,BUILDINGID,TYPEOFBUILDINGBLOCK,BASELEVEL,ROOFLEVEL,BASELEVELDATASOURCE,ROOFLEVELDATASOURCE,BUILDINGSTATUS,CERTAINTY,Shape_Length,Shape_Area,geometry
0,2024-05-08,1910247820,T,8.1,16.799999,4.0,4.0,E,1,23.908286,31.450706,"MULTIPOLYGON Z (((816145.174 831352.931 0.000,..."
1,2024-05-08,1910247815,T,8.1,16.100000,4.0,4.0,E,1,24.364307,31.712137,"MULTIPOLYGON Z (((816134.845 831345.483 0.000,..."
2,2024-05-07,1910247774,TS,NaN,NaN,NaN,NaN,E,1,16.514416,15.452368,"MULTIPOLYGON Z (((818016.014 838748.731 0.000,..."
3,2024-05-03,1910247696,TS,7.2,10.300000,4.0,4.0,E,1,32.868446,53.716489,"MULTIPOLYGON Z (((831625.175 840983.536 0.000,..."
4,2024-05-03,1910247695,TS,7.2,10.400000,4.0,4.0,E,1,121.792729,226.482654,"MULTIPOLYGON Z (((831631.172 840978.743 0.000,..."


In [8]:
# Select only the "BUILDINGID", "ENGLISHBUILDINGNAME", and "CHINESEBUILDINGNAME" columns from df3
df3_subset = df3[["BUILDINGID", "ENGLISHBUILDINGNAME", "CHINESEBUILDINGNAME"]]

# Merge gdf1 with the subset of df3 on the "BUILDINGID" column
gdf2 = pd.merge(gdf1, df3_subset, on="BUILDINGID")

# Display the first few rows of the merged DataFrame
gdf2.head()

,LASTUPDATEDATE,BUILDINGID,TYPEOFBUILDINGBLOCK,BASELEVEL,ROOFLEVEL,BASELEVELDATASOURCE,ROOFLEVELDATASOURCE,BUILDINGSTATUS,CERTAINTY,Shape_Length,Shape_Area,geometry,ENGLISHBUILDINGNAME,CHINESEBUILDINGNAME
0,2024-05-08,1910247693,T,5.3,120.0,10.0,10.0,E,1,134.739147,1175.061936,"MULTIPOLYGON Z (((834979.859 812118.914 0.000,...",Viva Place,偉華匯
1,2024-05-05,1910247673,P,NaN,NaN,NaN,NaN,P,1,576.375163,4219.597921,"MULTIPOLYGON Z (((838683.841 832108.401 0.000,...",Podium,平台
2,2024-05-05,1910247670,P,NaN,NaN,NaN,NaN,P,1,211.508289,1867.085492,"MULTIPOLYGON Z (((838802.481 832136.004 0.000,...",Podium,平台
3,2024-05-05,1910247669,P,NaN,NaN,NaN,NaN,P,1,267.948414,2513.780813,"MULTIPOLYGON Z (((838774.334 832051.217 0.000,...",Podium,平台
4,2024-05-05,1910247632,T,NaN,NaN,NaN,NaN,P,1,187.242046,1123.065147,"MULTIPOLYGON Z (((838755.868 832088.787 0.000,...",RESIDENTIAL RECREATIONAL FACILITIES,住客康樂設施


In [52]:
#shp_path = "data/building.shp"

#Convert the datetimes to strings
#gdf2['LASTUPDATEDATE'] = gdf2['LASTUPDATEDATE'].dt.strftime('%Y-%m-%d')

#gdf2.to_file(shp_path, encoding='utf-8')

C:\Users\Steve_Lau\AppData\Local\Temp\ipykernel_26052\1715028947.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf2.to_file(shp_path, encoding='utf-8')


In [9]:
# Keep only relevant columns
gdf3 = gdf2.loc[:, ("BUILDINGID", "TYPEOFBUILDINGBLOCK", "BASELEVEL", "ROOFLEVEL", "BUILDINGSTATUS", "ENGLISHBUILDINGNAME", "CHINESEBUILDINGNAME")]

gdf3.head(30)

,BUILDINGID,TYPEOFBUILDINGBLOCK,BASELEVEL,ROOFLEVEL,BUILDINGSTATUS,ENGLISHBUILDINGNAME,CHINESEBUILDINGNAME
0,1910247693,T,5.300000,120.000000,E,Viva Place,偉華匯
1,1910247673,P,NaN,NaN,P,Podium,平台
2,1910247670,P,NaN,NaN,P,Podium,平台
3,1910247669,P,NaN,NaN,P,Podium,平台
4,1910247632,T,NaN,NaN,P,RESIDENTIAL RECREATIONAL FACILITIES,住客康樂設施
5,1910247623,T,NaN,NaN,P,Scenic Tower 1,優景閣第１座
6,1910247622,T,NaN,NaN,P,SCENIC TOWER 6,優景閣第６座
7,1910247621,T,NaN,NaN,P,MARINA TOWER 1,漾景閣第１座
8,1910247620,T,NaN,NaN,P,MARINA TOWER 2,漾景閣第２座
9,1910247619,T,NaN,NaN,P,MARINA TOWER 3,漾景閣第３座
